# Cohere Setup

In [49]:
#! pip install cohere

# Import the required modules
import cohere

# Set up the Cohere client
api_key = 'doLqjOzGKiQxEmEyb2PhMeYa6qmVwfATPOyex606' # Paste your API key here. Remember to not share it publicly 
co = cohere.Client(api_key)

# Setup

In [134]:
#Load datasets from huggingface
from datasets import list_datasets, load_dataset

#Load other libraries, modules
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import time


# Dataset

In [16]:
dataset = load_dataset('financial_phrasebank', 'sentences_allagree')

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to C:/Users/dhava/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [101]:
#extracting data
X = dataset['train']['sentence']
y = dataset['train']['label']


In [102]:
#multi-label dataset
y.unique() 

array([1, 2, 0], dtype=int64)

In [127]:
#spliting data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=101)

In [133]:
#checking shapes
print('Test Shape: ', X_test.shape)

Test Shape:  (680,)


# Text Classification

Cohere provides three options to for text classification:
1. Few-shot classification with the Classify endpoint
2. Develop custom classifeier with the Embed endpoint
3. Model Finetuning

The details could be found from the following link: https://txt.cohere.ai/classify-three-options/

# Prepare Dataset for Classify Endpoint

Reference: https://docs.cohere.ai/reference/classify

To make a prediction, Classify uses the provided examples of text + label pairs as a reference. 

The dataset pairs (i.e., text, label) are called EXAMPLES. They provide context to the model.

**co.Classify** module has three main arguments: model_type, inputs (texts to be classified), examples (context)

If using a custom model, then examples are not required. In this case, as we are interested in classifying sentiment on a financial dataset, I will be using 'finance-sentiment' model. 


In [128]:
'''
#to use examples, apply the following lines of code.
from cohere.classify import Example

#create examples
examples = list()
for txt, lbl in zip(X_train.tolist(),y_train.tolist()):
    examples.append(Example(txt,lbl))
    
#change the model type to 'medium' or as per requirement
'''

"\n#to use examples, apply the following lines of code.\nfrom cohere.classify import Example\n\n#create examples\nexamples = list()\nfor txt, lbl in zip(X_train.tolist(),y_train.tolist()):\n    examples.append(Example(txt,lbl))\n    \n#change the model type to 'medium' or as per requirement\n"

In [124]:
# Perform classification
def classify_text(text,examples):
    classifications = co.classify(
        model='finance-sentiment',
        inputs=[text]
        )
    
    return classifications.classifications[0].prediction

In [140]:
# Generate classification predictions on the test dataset

y_pred = list()

for i in range(int(len(X_test)%100)+1):
    time.sleep(60)
    y_pred.extend(X_test[i*100:i*100+100].apply(classify_text, args=(examples,)).tolist())


In [142]:
# Evaluation Metrics

y_test_c = ["NEGATIVE" if b==0 else "POSITIVE" if b==2 else "NEUTRAL"for b in y_test]

accuracy = accuracy_score(y_test_c, y_pred)
f1 = f1_score(y_test_c, y_pred, average='weighted')

print(f'Accuracy: {100*accuracy:.2f}')
print(f'F1-score: {100*f1:.2f}')

Accuracy: 97.50
F1-score: 97.48


# Interpretation

The results are extremely promising!!!

It would be interesting to analyse the text where we have misclassifications. Of the 680 samples in the test set, 97.50 were correctly classified. 

In [154]:

mis_classes = [j for i,j in zip(y_pred,y_test_c) if i!=j]

mis_index = [i for i in range(len(y_test)) if y_test_c[i]!=y_pred[i]]

In [163]:
mis_df = pd.DataFrame()
mis_df['text'] = np.array(X_test.tolist())[mis_index]
mis_df['original_label'] = np.array(y_test_c)[mis_index]
mis_df['predicted_label'] = np.array(y_pred)[mis_index]

In [165]:
#dataframe for the misclassified text in the test set/
mis_df

,text,original_label,predicted_label
0,"Profitability ( EBIT % ) was 13.6 % , compared...",NEGATIVE,POSITIVE
1,Finnish business software group AffectoGenimap...,NEGATIVE,POSITIVE
2,Negotiations with representatives of the perso...,NEUTRAL,NEGATIVE
3,Net profit was 35.5 mln compared with 29.8 mln .,POSITIVE,NEUTRAL
4,Markets had been expecting a poor performance ...,POSITIVE,NEUTRAL
5,The company also said that in Poland a profita...,POSITIVE,NEUTRAL
6,According to CEO Matti Perkonoja of the parent...,NEGATIVE,NEUTRAL
7,Operating profit was EUR 1.6 mn in 2005 compar...,NEGATIVE,POSITIVE
8,"During the strike , Finnair estimates to incur...",NEGATIVE,NEUTRAL
9,The refining margin for the year was $ 13.39 -...,POSITIVE,NEUTRAL
